In [1]:
import pandas as pd
import numpy as np

In [2]:
comm_df = pd.read_csv("data/chicago_comm_areas_zip.csv")

In [4]:
house_df = pd.read_csv("data/zillow_neighborhoods_zip_280824.csv")

In [10]:
house_df = house_df.rename(columns = {"RegionName": "zip_code"})

In [12]:
house_df_merged = comm_df.merge(house_df, how = 'left', on = 'zip_code')

In [15]:
house_df_merged.head(1)

,area_number,community_area,zip_code,RegionID,SizeRank,RegionType,StateName,State,City,Metro,...,2023-10-31,2023-11-30,2023-12-31,2024-01-31,2024-02-29,2024-03-31,2024-04-30,2024-05-31,2024-06-30,2024-07-31
0,1,ROGERS PARK,60626,84628.0,844.0,zip,IL,IL,Chicago,"Chicago-Naperville-Elgin, IL-IN-WI",...,225450.913401,226769.716586,227521.097365,227463.9698,227248.389738,227658.648406,229416.446938,231755.717378,233263.878866,234189.989549


In [39]:
df_long = pd.melt(house_df_merged,
                  id_vars=['area_number', 'community_area', 'RegionID', 'SizeRank', 'zip_code', 'RegionType', 'StateName', 'State', 'City', 'Metro', 'CountyName'], 
                  var_name='Date', 
                  value_name='avg_price')

In [40]:
df_long.Date = pd.to_datetime(df_long.Date)

In [41]:
# get mean price per date

df_means = df_long.groupby(['area_number','community_area', 'Date']).mean().reset_index().drop(['RegionID', 'SizeRank', 'zip_code'], axis = 1)

In [42]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "../../secrets/gcp_token.json"

In [43]:
df_means.to_gbq(destination_table = "chicago_house_price.zillow_house_prices", 
              project_id='wagon-bootcamp-428814',
              if_exists="replace",
               table_schema=[{"name": "Date", "type": "DATE"}])